In [ ]:
import topology_interface_main as tim
import yaehmop_tools

input_POSCAR="FILENAME"

#fxn is working
kpoints=tim.generate_kpoints(nkx=10,nky=10,nkz=10,max_kx=1,max_ky=1)

#unknown fxns
input_yaehmop_filename=yaehmop_tools.create_yaehmop_from_poscar(input_POSCAR,kpoints)
output_yaehmop_filename=yaehmop_tools.run_yaehmop(input_yaehmop_filename) #subprocess.call("bind "+input_yaehmop_filname)

#fxn is working
c_,z2_= tim.get_topology(output_yaehmop_filename)
print(c_)
print(z2_)